## Understanding and Predicting Property Maintenance Fines

This project is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this project, your task is to predict whether a given blight ticket will be paid on time.

All data for this project has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this project.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this project is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this project, over 0.75 will recieve full points.
___

For this project, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [6]:
def blight_model():
    graffiti = {'GRAFFITI TICKET' : 1, 0: 0}
    add = pd.read_csv('readonly/addresses.csv', low_memory = False)
    latlon = pd.read_csv('readonly/latlons.csv', low_memory = False)
    geo_loc = add.merge(latlon, on = 'address', how = 'inner' )

    df = pd.read_csv('readonly/train.csv', low_memory = False, encoding = 'iso-8859-1', usecols= ['ticket_id', 
                                                                                        'agency_name',
                                                                                        'inspector_name',
                                                                                        'state',
                                                                                        'country',
                                                                                        'violation_code',
                                                                                        'judgment_amount',
                                                                                        'hearing_date',
                                                                                        'ticket_issued_date',
                                                                                        'compliance'])

    df_join = df.merge(geo_loc[['ticket_id','lat', 'lon']], on= 'ticket_id', how = 'inner' )
    df_join.dropna(subset = ['lat', 'lon','compliance'], inplace = True)
    df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'])
    df['hearing_date'] = pd.to_datetime(df['hearing_date'])
    df_join['duration'] = df['hearing_date'] - df['ticket_issued_date']
    df_join.drop(columns = ['hearing_date', 'ticket_issued_date'], inplace = True)
    col = list(df_join.columns)
    col.remove('compliance')
    df_join['agency_name'] = pd.Categorical(df_join['agency_name'], ordered=True)
    df_join['agency_name'] = df_join['agency_name'].cat.codes
    df_join['country'] = pd.Categorical(df_join['country'], ordered=True)
    df_join['country'] = df_join['country'].cat.codes
    df_join['duration'] = [pd.Timedelta(x).days for x in df_join['duration']]
    df_join.fillna(72,inplace = True)
    def convert(col):
        c = {'inspector_name': 378, 'violation_code': 8, 'state': 96}
        count = df_join[col].value_counts()
        categoriesToKeep = count[count>c[col]].index.tolist()
        df_join[col] = pd.Categorical(df_join[col], categories = categoriesToKeep,
                               ordered = True)
        df_join[col] = df_join[col].cat.add_categories('<unknown>').fillna('<unknown>')
        df_join[col] = df_join[col].cat.codes

        test[col] = pd.Categorical(test[col],categories = categoriesToKeep,
                                  ordered = True)
        test[col] = test[col].cat.add_categories('<unknown>').fillna('<unknown>')
        test[col] = test[col].cat.codes

    test = pd.read_csv('readonly/test.csv', low_memory = False, encoding = 'iso-8859-1', usecols=['ticket_id', 
                                                                                        'agency_name',
                                                                                        'inspector_name',
                                                                                        'state',
                                                                                        'country',
                                                                                        'violation_code',
                                                                                        'judgment_amount',
                                                                                        'hearing_date',
                                                                                        'ticket_issued_date',
                                                                                        ])
    test = test.merge(geo_loc[['ticket_id', 'lat', 'lon']], on  = 'ticket_id', how = 'inner')

    test['lat'].fillna(42.392626, inplace = True)
    test['lon'].fillna(-83.117658, inplace = True)

    test['duration'] = pd.to_datetime(test['hearing_date']) - pd.to_datetime(test['ticket_issued_date'])
    test.drop(columns = ['ticket_issued_date','hearing_date'], inplace = True)
    test['duration'] = [pd.Timedelta(x).days for x in test['duration']]
    test.fillna(72, inplace = True)
    test['agency_name'] = pd.Categorical(test['agency_name'], ordered=True)
    test['agency_name'] = test['agency_name'].cat.codes
    test['country'] = pd.Categorical(test['country'], ordered=True)
    test['country'] = test['country'].cat.codes

    convert('inspector_name')
    convert('violation_code')
    convert('state')

    from sklearn.model_selection import train_test_split
    X = df_join[col].copy()
    y = df_join['compliance'].copy()

    x_train, x_test, y_train, y_test = train_test_split(X,y, random_state = 0, shuffle = True )
    rfc = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=24, verbose=0, warm_start=False)
    rfc.fit(x_train,y_train)
    result = rfc.predict_proba(test)
    r = pd.DataFrame(result, columns = rfc.classes_, index = test['ticket_id'])
    r = pd.Series(r[1.0])
    return r

In [3]:
blight_model()

ticket_id
284932    0.146658
285362    0.040837
285361    0.142139
285338    0.112124
285346    0.153722
            ...   
376496    0.046134
376497    0.046134
376499    0.154450
376500    0.154450
369851    0.362004
Name: 1.0, Length: 61001, dtype: float64

In [ ]:
#sanity test
bm = blight_model()
res = '{:40s}'.format('Object Type:')
res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
res += '{:40s}'.format('Data Shape:')
res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
res += '{:40s}'.format('Data Values Type:')
res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
res += '{:40s}'.format('Data Values Infinity:')
res += ['Failed: values should not be infinity\n','Passed\n'][not any(np.isinf(bm))]
res += '{:40s}'.format('Data Values NaN:')
res += ['Failed: values should not be NaN\n','Passed\n'][not any(np.isnan(bm))]
res += '{:40s}'.format('Data Values in [0,1] Range:')
res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
res += '{:40s}'.format('Data Values not all 0 or 1:')
res += ['Failed: values should be scores not predicted labels\n','Passed\n'][not all((bm.isin({0,1,0.0,1.0})))]
res += '{:40s}'.format('Index Type:')
res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
res += '{:40s}'.format('Index Values:')
if bm.index.shape==(61001,):
    res +=['Failed: index values should match test.csv\n','Passed\n'
          ][all(pd.read_csv('test.csv',usecols=[0],index_col=0
                           ).sort_index().index.values==bm.sort_index().index.values)]
else:
    res+='Failed: bm.index length should be 61001'
res += '{:40s}'.format('Can run model twice:')
bm2 = None
try:
    bm2 = blight_model()
    res += 'Passed\n'
except:
    res += ['Failed: second run of blight_model() threw an Exception']
res += '{:40s}'.format('Can run model twice with same results:')
if not bm2 is None:
    res += ['Failed: second run of blight_model() produced different results (this might not be a problem)\n','Passed\n'][
        all(bm.apply(lambda x:round(x,3))==bm2.apply(lambda x:round(x,3))) and all(bm.index==bm2.index)]    
print(res)

## Columns dropped:
* disposition: all say responsible, no information gain
* violation_code is sufficiet no need for violation_description
* violation_zip_code has only nan values in train but some non nan in test, better to use lon_lat data
* non_us_str_code is redundant column with many nan values, better to use country column
* grafitti_status column is manually oneHot encoded to graffiti column
* violator_name, has 84656 categories whereas 159k rows in data
* violation_street_name: number is sufficient
* mailing_street_name: number is sufficient
* fine_amount 
* admin_fee 
* state_fee 
* late_fee 
* discount_amount 
* clean_up_cost: judgment cost is their addition so no problem there
* ticket_issued_date
* hearing_date: added duration instead of them
* city: many cities exist in test data nearly 2k, but not in training data, no benefit in generalization
* mailing_address_stree number: same reason as above
* grafitti_status: all 0 for training data, no training done, meaningless

In [7]:
graffiti = {'GRAFFITI TICKET' : 1, 0: 0}
add = pd.read_csv('addresses.csv', low_memory = False)
latlon = pd.read_csv('latlons.csv', low_memory = False)
geo_loc = add.merge(latlon, on = 'address', how = 'inner' )

df = pd.read_csv('train.csv', low_memory = False, encoding = 'iso-8859-1', usecols= ['ticket_id', 
                                                                                    'agency_name',
                                                                                    'inspector_name',
                                                                                    'state',
                                                                                    'country',
                                                                                    'violation_code',
                                                                                    'judgment_amount',
                                                                                    'hearing_date',
                                                                                    'ticket_issued_date',
                                                                                    'compliance'])

df_join = df.merge(geo_loc[['ticket_id','lat', 'lon']], on= 'ticket_id', how = 'inner' )
df_join.dropna(subset = ['lat', 'lon','compliance'], inplace = True)
df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'])
df['hearing_date'] = pd.to_datetime(df['hearing_date'])
df_join['duration'] = df['hearing_date'] - df['ticket_issued_date']
df_join.drop(columns = ['hearing_date', 'ticket_issued_date'], inplace = True)
col = list(df_join.columns)
col.remove('compliance')
df_join['agency_name'] = pd.Categorical(df_join['agency_name'], ordered=True)
df_join['agency_name'] = df_join['agency_name'].cat.codes
df_join['country'] = pd.Categorical(df_join['country'], ordered=True)
df_join['country'] = df_join['country'].cat.codes
df_join['duration'] = [pd.Timedelta(x).days for x in df_join['duration']]
df_join.fillna(72,inplace = True)
def convert(col):
    c = {'inspector_name': 378, 'violation_code': 8, 'state': 96}
    count = df_join[col].value_counts()
    categoriesToKeep = count[count>c[col]].index.tolist()
    df_join[col] = pd.Categorical(df_join[col], categories = categoriesToKeep,
                           ordered = True)
    df_join[col] = df_join[col].cat.add_categories('<unknown>').fillna('<unknown>')
    df_join[col] = df_join[col].cat.codes
    
    test[col] = pd.Categorical(test[col],categories = categoriesToKeep,
                              ordered = True)
    test[col] = test[col].cat.add_categories('<unknown>').fillna('<unknown>')
    test[col] = test[col].cat.codes

test = pd.read_csv('test.csv', low_memory = False, encoding = 'iso-8859-1', usecols=['ticket_id', 
                                                                                    'agency_name',
                                                                                    'inspector_name',
                                                                                    'state',
                                                                                    'country',
                                                                                    'violation_code',
                                                                                    'judgment_amount',
                                                                                    'hearing_date',
                                                                                    'ticket_issued_date',
                                                                                    ])
test = test.merge(geo_loc[['ticket_id', 'lat', 'lon']], on  = 'ticket_id', how = 'inner')

test['lat'].fillna(42.392626, inplace = True)
test['lon'].fillna(-83.117658, inplace = True)

test['duration'] = pd.to_datetime(test['hearing_date']) - pd.to_datetime(test['ticket_issued_date'])
test.drop(columns = ['ticket_issued_date','hearing_date'], inplace = True)
test['duration'] = [pd.Timedelta(x).days for x in test['duration']]
test.fillna(72, inplace = True)
test['agency_name'] = pd.Categorical(test['agency_name'], ordered=True)
test['agency_name'] = test['agency_name'].cat.codes
test['country'] = pd.Categorical(test['country'], ordered=True)
test['country'] = test['country'].cat.codes

convert('inspector_name')
convert('violation_code')
convert('state')

from sklearn.model_selection import train_test_split
X = df_join[col].copy()
y = df_join['compliance'].copy()

x_train, x_test, y_train, y_test = train_test_split(X,y, random_state = 0, shuffle = True )

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
#sample for fast compoutations
x_train_s = x_train[:20000]
y_train_s = y_train[:20000]

In [ ]:
#rfc = RandomForestClassifier(random_state = 24)
#param_grid = { 
#    'n_estimators': [200, 500],
#    'max_features': ['auto', 'sqrt', 'log2'],
#    'max_depth' : [4,5,6,7,8],
#    'criterion' :['gini', 'entropy']
#}

#CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, scoring = 'roc_auc')
#CV_rfc.fit(x_train_s, y_train_s)
#print(CV_rfc.best_params_)

#rfc1 = RandomForestClassifier(random_state = 24, criterion = 'entropy', max_depth = 8, max_features = 'auto', n_estimators = 500) #add params according to GS result
#rfc1.fit(x_train, y_train) #get model summary from here and paste in development code

In [ ]:
rfc = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=24, verbose=0, warm_start=False)

In [ ]:
rfc.fit(x_train, y_train)

In [ ]:
result = rfc.predict_proba(test)
r = pd.DataFrame(result, columns = rfc.classes_, index = test['ticket_id'])
r = pd.Series(r[1.0])

In [ ]:
def blight_model():
    
    add = pd.read_csv('addresses.csv', low_memory = False)
    latlon = pd.read_csv('latlons.csv', low_memory = False)
    geo_loc = add.merge(latlon, on = 'address', how = 'inner' )

    df = pd.read_csv('train.csv', low_memory = False, encoding = 'iso-8859-1', usecols= ['ticket_id', 
                                                                                        'agency_name',
                                                                                        'inspector_name',
                                                                                        'state',
                                                                                        'country',
                                                                                        'violation_code',
                                                                                        'judgment_amount',
                                                                                        'hearing_date',
                                                                                        'ticket_issued_date',
                                                                                        'compliance'])

    df_join = df.merge(geo_loc[['ticket_id','lat', 'lon']], on= 'ticket_id', how = 'inner' )
    df_join.dropna(subset = ['lat', 'lon','compliance'], inplace = True)
    df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'])
    df['hearing_date'] = pd.to_datetime(df['hearing_date'])
    df_join['duration'] = df['hearing_date'] - df['ticket_issued_date']
    df_join.drop(['hearing_date', 'ticket_issued_date'], inplace = True, axis = 1)
    col = list(df_join.columns)
    col.remove('compliance')
    df_join['agency_name'] = pd.Categorical(df_join['agency_name'], ordered=True)
    df_join['agency_name'] = df_join['agency_name'].cat.codes
    df_join['country'] = pd.Categorical(df_join['country'], ordered=True)
    df_join['country'] = df_join['country'].cat.codes
    df_join['duration'].fillna(72, inplace = True)
    df_join['duration'] = [pd.Timedelta(x).days for x in df_join['duration']]
    df_join.fillna(72,inplace = True)
    def convert(col):
        c = {'inspector_name': 378, 'violation_code': 8, 'state': 96}
        count = df_join[col].value_counts()
        categoriesToKeep = count[count>c[col]].index.tolist()
        df_join[col] = pd.Categorical(df_join[col], categories = categoriesToKeep,
                               ordered = True)
        df_join[col] = df_join[col].cat.add_categories('<unknown>').fillna('<unknown>')
        df_join[col] = df_join[col].cat.codes

        test[col] = pd.Categorical(test[col],categories = categoriesToKeep,
                                  ordered = True)
        test[col] = test[col].cat.add_categories('<unknown>').fillna('<unknown>')
        test[col] = test[col].cat.codes

    test = pd.read_csv('test.csv', low_memory = False, encoding = 'iso-8859-1', usecols=['ticket_id', 
                                                                                        'agency_name',
                                                                                        'inspector_name',
                                                                                        'state',
                                                                                        'country',
                                                                                        'violation_code',
                                                                                        'judgment_amount',
                                                                                        'hearing_date',
                                                                                        'ticket_issued_date',
                                                                                        ])
    test = test.merge(geo_loc[['ticket_id', 'lat', 'lon']], on  = 'ticket_id', how = 'inner')

    test['lat'].fillna(42.392626, inplace = True)
    test['lon'].fillna(-83.117658, inplace = True)


    test['duration'] = pd.to_datetime(test['hearing_date']) - pd.to_datetime(test['ticket_issued_date'])
    test.drop(['ticket_issued_date','hearing_date'], inplace = True, axis = 1)
    test['duration'].fillna(72, inplace = True)
    test['duration'] = [pd.Timedelta(x).days for x in test['duration']]
    test['agency_name'] = pd.Categorical(test['agency_name'], ordered=True)
    test['agency_name'] = test['agency_name'].cat.codes
    test['country'] = pd.Categorical(test['country'], ordered=True)
    test['country'] = test['country'].cat.codes

    convert('inspector_name')
    convert('violation_code')
    convert('state')

    from sklearn.model_selection import train_test_split
    X = df_join[col].copy()
    y = df_join['compliance'].copy()

    x_train, x_test, y_train, y_test = train_test_split(X,y, random_state = 0 )
    rfc = RandomForestClassifier(bootstrap=True, criterion='entropy',
            max_depth=8, max_features='auto', min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, 
            random_state=24, verbose=0)
    rfc.fit(x_train,y_train)

    result = rfc.predict_proba(test)
    r = pd.DataFrame(result, columns = rfc.classes_, index = test['ticket_id'])
    r = pd.Series(r[1.0])
    return r